# Task L14
## Import the required libraries

In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Loading and preparing data

In [26]:
df = pd.read_csv("creditcard.csv").drop(columns=["Time"])
features = ['V10','V14','V4','V12', 'V11','V17','Class']
df = df[features].copy()

X = df.drop(columns=['Class']).values
y = df['Class'].values

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Scaling

In [27]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# List of methods

In [ ]:
methods = {
    "Isolation Forest": IsolationForest(contamination=0.01, random_state=42),
    "Local Outlier Factor": LocalOutlierFactor(contamination=0.01, novelty=False),
    "Elliptic Envelope": EllipticEnvelope(contamination=0.01, random_state=42),
    "One-Class SVM": OneClassSVM(nu=0.01, kernel='rbf', gamma='scale'),
    "DBSCAN": DBSCAN(eps=0.5, min_samples=5)
}

results = []

for name, model in methods.items():
    print(f"\n=== {name} ===")
    if name == "Local Outlier Factor":
        y_pred = model.fit_predict(X_scaled)
    elif name == "DBSCAN":
        y_pred = model.fit_predict(X_scaled)
    else:
        model.fit(X_scaled)
        y_pred = model.predict(X_scaled)

    y_pred = np.where(y_pred == -1, 1, 0)

    mask = ~np.isnan(y)
    y_true_clean = y[mask]
    y_pred_clean = y_pred[mask]

    assert len(y_true_clean) == len(y_pred_clean), f"len(y_true)={len(y_true_clean)}, len(y_pred)={len(y_pred_clean)}"

    report = classification_report(y_true_clean, y_pred_clean, output_dict=True, zero_division=0)
    cm = confusion_matrix(y_true_clean, y_pred_clean)

    print("Classification Report:")
    print(classification_report(y_true_clean, y_pred_clean, zero_division=0))
    print("Confusion Matrix:")
    print(pd.DataFrame(cm,
                       index=["true_non-fraud", "true_fraud"],
                       columns=["pred_non-fraud", "pred_fraud"]))

    fraud_metrics = report.get("1", {"precision": 0, "recall": 0, "f1-score": 0, "support": 0})
    results.append({
        "Model": name,
        "Precision": fraud_metrics["precision"],
        "Recall": fraud_metrics["recall"],
        "F1-Score": fraud_metrics["f1-score"],
        "Support": fraud_metrics["support"]
    })


=== Isolation Forest ===
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    284315
           1       0.15      0.85      0.25       492

    accuracy                           0.99    284807
   macro avg       0.57      0.92      0.62    284807
weighted avg       1.00      0.99      0.99    284807

Confusion Matrix:
                pred_non-fraud  pred_fraud
true_non-fraud          281883        2432
true_fraud                  75         417

=== Local Outlier Factor ===


/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_lof.py:322: UserWarning: Duplicate values are leading to incorrect results. Increase the number of neighbors for more accurate results.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    284315
           1       0.00      0.00      0.00       492

    accuracy                           0.99    284807
   macro avg       0.50      0.49      0.50    284807
weighted avg       1.00      0.99      0.99    284807

Confusion Matrix:
                pred_non-fraud  pred_fraud
true_non-fraud          281466        2849
true_fraud                 492           0

=== Elliptic Envelope ===
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    284315
           1       0.14      0.82      0.24       492

    accuracy                           0.99    284807
   macro avg       0.57      0.91      0.62    284807
weighted avg       1.00      0.99      0.99    284807

Confusion Matrix:
                pred_non-fraud  pred_fraud
true_non-fraud          281871        2444
true_fraud            

# Building a results table

In [ ]:
df_results = pd.DataFrame(results)
print("\n=== Summary ===")
print(df_results)